In [1]:
!pip install bitmex

https://medium.com/swlh/retrieving-full-historical-data-for-every-cryptocurrency-on-binance-bitmex-using-the-python-apis-27b47fd8137f

In [1]:
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
import tqdm

In [2]:
### API
bitmex_api_key = 'PUNPhX0cNSinQ7Zv4TRAETdx'    #Enter your own API-key here
bitmex_api_secret = 'duc5b9JxLy68Oza_9cfv6I8HztwaYFSOdETWmOYOCojFtGv7' #Enter your own API-secret here



In [3]:
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
bitmex_client = bitmex(test=True, api_key=bitmex_api_key, api_secret=bitmex_api_secret)

/home/james/anaconda3/lib/python3.8/site-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [6]:
### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  
        old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "bitmex": 
        old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "bitmex": 
        new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new



def get_all_bitmex(symbol, kline_size, start_datetime, end_datetime, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
    else: 
        data_df = pd.DataFrame()
    
    #start_datetime, end_datetime = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    
    delta_min = (end_datetime - start_datetime).total_seconds() / 60 # number of mins
    
    available_data = math.ceil(delta_min/binsizes[kline_size])
    
    
    rounds = math.ceil(available_data / batch_size)
    
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (start_datetime + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    
    data_df.set_index('timestamp', inplace=True)
    
    if save and rounds > 0: 
        data_df.to_csv(filename)
    print('All caught up..!')
    
    return data_df

In [8]:
start_datetime = datetime(2020, 6, 1, 0, 0, 0, 0)
end_datetime = datetime(2020, 6, 30, 23, 59, 59, 0)


bitmex_symbols = ["XBTUSD"]
for symbol in bitmex_symbols:
    data = get_all_bitmex(symbol, '5m', start_datetime, end_datetime, save = True)

<ipython-input-6-9ef81dd9da8e>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for round_num in tqdm_notebook(range(rounds)):



All caught up..!


In [9]:
data

,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2020-06-01 00:00:00+00:00,XBTUSD,9495.5,9564.0,9422.5,9485.5,869,1373409,9484.0668,66,14481459203,144.814592,1373409.0
2020-06-01 00:05:00+00:00,XBTUSD,9485.5,9486.0,9391.0,9407.5,582,453921,9417.9695,30,4819790425,48.197904,453921.0
2020-06-01 00:10:00+00:00,XBTUSD,9407.5,9448.0,9407.5,9423.0,401,92576,9426.8477,24,982062412,9.820624,92576.0
2020-06-01 00:15:00+00:00,XBTUSD,9423.0,9494.0,9422.5,9461.5,364,98877,9467.9038,100,1044383093,10.443831,98877.0
2020-06-01 00:20:00+00:00,XBTUSD,9461.5,9474.0,9461.0,9464.0,87,7485,9464.3195,63,79091193,0.790912,7485.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-02 05:35:00+00:00,XBTUSD,9226.5,9226.5,9226.5,9226.5,24,1325,9227.0000,65,14360350,0.143603,1325.0
2020-07-02 05:40:00+00:00,XBTUSD,9226.5,9232.5,9226.0,9226.0,58,5795,9228.4976,5,62794990,0.627950,5795.0
2020-07-02 05:45:00+00:00,XBTUSD,9226.0,9232.5,9226.0,9226.0,43,5114,9227.6460,1,55420945,0.554209,5114.0
